In [1]:
# Data processing
import iris
import numpy as np
import xarray as xr
import dateutil.parser
import warnings
warnings.filterwarnings('ignore', module='iris')
from iris.experimental import stratify
from iris.analysis import trajectory
from pathlib import Path
from datetime import datetime, timedelta
## Scientific parameters
M_air = 28.97 # molar mass of dry air [g mol-1]
# Coefficients to convert mass mixing ratio to volume mixing ratio
coeff_c2h6 = (M_air/30.0690)*1e12
coeff_c3h8 = (M_air/44.0956)*1e12
coeff_meono2 = (M_air/77.0394)*1e12
coeff_etono2 = (M_air/91.0660)*1e12
coeff_nprono2 = (M_air/105.0926)*1e12
coeff_iprono2 = (M_air/105.0926)*1e12

#### Read data

In [2]:
path_to_atom = Path('../../../obs/ATom/nc/data')
# Select flight
fname_atom_flight = 'MER-WAS_DC8_20160729_R14.nc'
# Read ATom data
atom_dsinf = xr.open_dataset(path_to_atom / fname_atom_flight, decode_times=True)
atom_dsmms = xr.open_dataset(path_to_atom / fname_atom_flight, group='MMS', decode_cf=True)

In [3]:
# Choose UKCA run
ukca_run_name = 'xojnc'
path_to_ukca = Path('../../../../data') / ukca_run_name
fname_ukca_run = f'{ukca_run_name}_chem.pp'
# Read UKCA data
# cb_c2h6 = iris.load_cube(str(path_to_ukca / fname_ukca_run), 'mass_fraction_of_ethane_in_air')*coeff_c2h6
# cb_c3h8 = iris.load_cube(str(path_to_ukca / fname_ukca_run), 'mass_fraction_of_propane_in_air')*coeff_c3h8
# cb_meono2 = iris.load_cube(str(path_to_ukca / fname_ukca_run), 'mass_fraction_of_methyl_nitrate_in_air')*coeff_meono2
# cb_etono2 = iris.load_cube(str(path_to_ukca / fname_ukca_run), iris.AttributeConstraint(STASH='m01s34i096'))*coeff_etono2
# cb_nprono2 = iris.load_cube(str(path_to_ukca / fname_ukca_run), iris.AttributeConstraint(STASH='m01s34i097'))*coeff_nprono2
# cb_iprono2 = iris.load_cube(str(path_to_ukca / fname_ukca_run), iris.AttributeConstraint(STASH='m01s34i098'))*coeff_iprono2

In [18]:
# Extract datetimes from ATom and UKCA data
atom_datetime = atom_dsinf.time.values.astype('<M8[us]').astype(datetime)
atom_date_strt = atom_datetime[0]
atom_date_stop = atom_datetime[-1]
ukca_days_since = dateutil.parser.parse(str(cb_nc4h10.coord('time').units)[12:])
_ukca_datetime = []
for i, ii in enumerate(list(cb_nc4h10.coord('time').points)):
    _ukca_datetime.append(ukca_days_since + timedelta(hours=float(f'{ii:1.3f}'))) # UKCA's 'hours since 1970-01-01' format is 1.3f
ukca_datetime = np.array(_ukca_datetime, dtype=datetime)
# Create a common timestamp/flight merge identifier for trajectories
compared_common_id = f'{atom_date_strt.strftime("%y%m%d")}_{atom_date_stop.strftime("%y%m%d")}_{ukca_datetime[0].strftime("%y%m")}_singl'
print(compared_common_id)

160729_160729_1607_singl


#### Process data
##### 1. Relevel model data from level height to absolute height 

In [ ]:
# Relevel model data from level height to target levels
tgt_levels = np.arange(0, 14000, 100) # [meters]
cbr_c2h6 = stratify.relevel(cb_c2h6, cb_c2h6.coord('altitude'), tgt_levels, axis='level_height')
cbr_c3h8 = stratify.relevel(cb_c3h8, cb_c3h8.coord('altitude'), tgt_levels, axis='level_height')
cbr_meono2 = stratify.relevel(cb_meono2, cb_meono2.coord('altitude'), tgt_levels, axis='level_height')
cbr_etono2 = stratify.relevel(cb_etono2, cb_etono2.coord('altitude'), tgt_levels, axis='level_height')
cbr_nprono2 = stratify.relevel(cb_nprono2, cb_nprono2.coord('altitude'), tgt_levels, axis='level_height')
cbr_iprono2 = stratify.relevel(cb_iprono2, cb_iprono2.coord('altitude'), tgt_levels, axis='level_height')

##### 2. Calculate trajectories

In [21]:
# Reconstruct a flight trajectory from ATom geospacial and datetime data
if compared_common_id == '160803_160803_1608_singl':
    # Because ATom flight on 20160803 has one lat, lon, alt value equal to 'nan', have to interpolate between adjacent cells
    # to replace 'nan' with an interpolated value.
    sample_lats = atom_dsmms.G_LAT.interpolate_na(dim='time', method='linear').data
    sample_lons = atom_dsmms.G_LONG.interpolate_na(dim='time', method='linear').data+360
    sample_alts = atom_dsmms.G_ALT.interpolate_na(dim='time', method='linear').data
else:
    sample_lats = atom_dsmms.G_LAT.data 
    sample_lons = atom_dsmms.G_LONG.data+360
    sample_alts = atom_dsmms.G_ALT.data
sample_lats_lons_alts = [('latitude', sample_lats), ('longitude', sample_lons), ('altitude', sample_alts)]

In [27]:
# Interpolate releveled model data to flight trajectory
traj_c2h6 = trajectory.interpolate(cbr_c2h6, sample_lats_lons_alts)
traj_c3h8 = trajectory.interpolate(cbr_c3h8, sample_lats_lons_alts)
traj_meono2 = trajectory.interpolate(cbr_meono2, sample_lats_lons_alts)
traj_etono2 = trajectory.interpolate(cbr_etono2, sample_lats_lons_alts)
traj_nprono2 = trajectory.interpolate(cbr_nprono2, sample_lats_lons_alts)
traj_iprono2 = trajectory.interpolate(cbr_iprono2, sample_lats_lons_alts)

##### 3. Extract cross sections

In [28]:
# Extract a cross section from model data along flight track
sample_lats_lons = [('latitude', sample_lats), ('longitude', sample_lons)]
crsc_c2h6 = trajectory.interpolate(cbr_c2h6, sample_lats_lons)
crsc_c3h8 = trajectory.interpolate(cbr_c3h8, sample_lats_lons)
crsc_meono2 = trajectory.interpolate(cbr_meono2, sample_lats_lons)
crsc_etono2 = trajectory.interpolate(cbr_etono2, sample_lats_lons)
crsc_nprono2 = trajectory.interpolate(cbr_nprono2, sample_lats_lons)
crsc_iprono2 = trajectory.interpolate(cbr_iprono2, sample_lats_lons)

#### Save processed data

In [29]:
path_to_prcd = Path('../processed') / ukca_run_name

In [31]:
# Save trajectories to .nc
iris.save(traj_c2h6, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_c2h6.nc')
iris.save(traj_c3h8, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_c3h8.nc')
iris.save(traj_meono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_meono2.nc')
iris.save(traj_etono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_etono2.nc')
iris.save(traj_nprono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_nprono2.nc')
iris.save(traj_iprono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_iprono2.nc')

In [32]:
# Save cross sections to .nc
iris.save(crsc_c2h6, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_c2h6.nc')
iris.save(crsc_c3h8, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_c3h8.nc')
iris.save(crsc_meono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_meono2.nc')
iris.save(crsc_etono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_etono2.nc')
iris.save(crsc_nprono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_nprono2.nc')
iris.save(crsc_iprono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_iprono2.nc')